In [ ]:
import project_lib as mylib
import numpy as np
import matplotlib.pyplot as plt

In this file I just want to check if I can build the multibody hamiltonian as it is in Neil's paper correctly

In [ ]:
# in this cell I am trying to build the neutral hamiltonian that is mentioned in the circuits paper
# here I define 4 logical qubits and 4 ancilla qubits
# I modified this to be correct from what it previously was. I was getting the ancilla bit assignments the wrong way round
# I was putting the 0s on the right hand side instead of the left
log_qub = 4
anc_qub = log_qub
qubits = log_qub*2

# JN<<q0<Ja;   J = Ja

# I will use these parameters later
JN = 0.1
q0 = 50.
Ja = 100.
Jl = Ja

# construct h
hl = np.asarray([-Ja+q0]*log_qub) # h for the logical qubits
ha = np.asarray([-Ja*(2*i-log_qub)+q0 for i in range(1,log_qub+1)]) # h for the ancilla qubits
h = np.concatenate((hl,ha)) # the full h

# constuct J
J_log_log = Jl*np.triu(np.ones((log_qub,log_qub)), k=1) # the J component for the logical qubit interactions
J_log_anc = Ja*np.ones((log_qub,log_qub)) # interactions between logical and ancilla bits
Jt = np.concatenate((J_log_log,J_log_anc),axis=1) # make top half of J array
Jb = np.zeros(Jt.shape) # make bottom half of J array
J = np.concatenate((Jt,Jb),axis=0) # the full J

# make vectors for the base states with the ancilla qubits in their ground states
lbo1 = [0,0,0,1]
ab = [0]*(sum(lbo1)) + [1]*(len(lbo1)-sum(lbo1))
bitso1 = lbo1+ab
vo1 = mylib.make_eigen_vector(bitso1)

lbo2 = [0,1,1,1]
ab = [0]*(sum(lbo2)) + [1]*(len(lbo2)-sum(lbo2))
bitso2 = lbo2+ab
vo2 = mylib.make_eigen_vector(bitso2)

lbe1 = [0,0,0,0]
ab = [0]*(sum(lbe1)) + [1]*(len(lbe1)-sum(lbe1))
bitse1 = lbe1+ab
ve1 = mylib.make_eigen_vector(bitse1)

lbe2 = [0,0,1,1]
ab = [0]*(sum(lbe2)) + [1]*(len(lbe2)-sum(lbe2))
bitse2 = lbe2+ab
ve2 = mylib.make_eigen_vector(bitse2)

lbe3 = [1,1,1,1]
ab = [0]*(sum(lbe3)) + [1]*(len(lbe3)-sum(lbe3))
bitse3 = lbe3+ab
ve3 = mylib.make_eigen_vector(bitse3)

# make anneal object
a = mylib.Anneal(qubits,[h,J],T=100,points = 100)


In [ ]:
# test for neutrality
print vo1*a.Hp*vo1.getH(), vo2*a.Hp*vo2.getH()
print ve1*a.Hp*ve1.getH(), ve2*a.Hp*ve2.getH(), ve3*a.Hp*ve3.getH()

In [ ]:
# test eq[11] in circuits paper
# These parameters turn the hamiltonian into just a version of  this equation
h = np.concatenate((hl,np.zeros(hl.shape)))*(-1./50)
J =np.zeros((qubits,qubits))
a = mylib.Anneal(qubits,[h,J],T=100,points = 100)

def f1(lb):
    return float(2*np.sum(lb) - len(lb))

print (vo1*a.Hp*vo1.getH()).item(0), (vo2*a.Hp*vo2.getH()).item(0)
print f1(lbo1), f1(lbo2)
print (ve1*a.Hp*ve1.getH()).item(0), (ve2*a.Hp*ve2.getH()).item(0), (ve3*a.Hp*ve3.getH()).item(0)
print f1(lbe1), f1(lbe2), f1(lbe3)

In [ ]:
# test eq[12] in circuits paper
# these parameters are to match
h = np.concatenate((np.zeros(hl.shape),np.zeros(hl.shape)))
J_12 = np.triu(np.ones((log_qub,log_qub)), k=1)
Jt = np.concatenate((J_12,np.zeros(J_12.shape)),axis=1)
Jb = np.zeros(Jt.shape)
J = np.concatenate((Jt,Jb),axis=0)
a = mylib.Anneal(qubits,[h,J],T=100,points = 100)

def f2(lb):
    return 0.5*(2*np.sum(lb) - len(lb))**2-0.5*len(lb)

print (vo1*a.Hp*vo1.getH()).item(0), (vo2*a.Hp*vo2.getH()).item(0)
print f2(lbo1), f2(lbo2)
print (ve1*a.Hp*ve1.getH()).item(0), (ve2*a.Hp*ve2.getH()).item(0), (ve3*a.Hp*ve3.getH()).item(0)
print f2(lbe1), f2(lbe2), f2(lbe3)

In [ ]:
# test eq[13] in circuits paper
# these parameters are to match
h = np.concatenate((np.zeros(hl.shape),np.zeros(hl.shape)))
J_log_log = np.zeros((log_qub,log_qub))
J_log_anc = np.ones((log_qub,log_qub))
Jt = np.concatenate((J_log_log,J_log_anc),axis=1)
Jb = np.zeros(Jt.shape)
J = np.concatenate((Jt,Jb),axis=0)
a = mylib.Anneal(qubits,[h,J],T=100,points = 100)

def f3(lb):
    '''This is Nick's version of equation 13'''
    return float(-(2*np.sum(lb) - len(lb))**2-2*np.sum(lb))
def f3_2(lb):
    '''This is the result from my derivation of eq 13'''
    return float(-4.*np.sum(lb)**2 + 4.*np.sum(lb)*len(lb) -len(lb)**2)
def f3_3(lb):
    '''This is my derivation plus the extra term difference between mine and nicks'''
    return f3_2(lb)-2.*np.sum(lb)

print 'Nup=1','Nup=3'
print (vo1*a.Hp*vo1.getH()).item(0), (vo2*a.Hp*vo2.getH()).item(0), ' hamiltonian'
print f3_2(lbo1), f3_2(lbo2), ' my derived function'
print f3(lbo1), f3(lbo2), ' nicks function'
print f3_3(lbo1), f3_3(lbo2), ' my f + extra bit', '\n'

print'Nup=0','Nup=2','Nup=4'
print (ve1*a.Hp*ve1.getH()).item(0), (ve2*a.Hp*ve2.getH()).item(0), (ve3*a.Hp*ve3.getH()).item(0), ' hamiltonian '
print f3_2(lbe1), f3_2(lbe2), f3_2(lbe3), ' my function'
print f3(lbe1), f3(lbe2), f3(lbe3), ' nicks function'
print f3_3(lbe1), f3_3(lbe2), f3_3(lbe3), 'my f + extra '

In [ ]:
# test eq[14] in circuits paper
# paramters here are to match this
Ja = 2.
q0 = 0.55
hl = np.zeros(log_qub)
ha = np.asarray([-Ja*(2*i-log_qub)+q0 for i in range(1,log_qub+1)])
h = np.concatenate((hl,ha))

J_log_log = np.zeros((log_qub,log_qub))
J_log_anc = np.zeros((log_qub,log_qub))
Jt = np.concatenate((J_log_log,J_log_anc),axis=1)
Jb = np.zeros(Jt.shape)
J = np.concatenate((Jt,Jb),axis=0)
a = mylib.Anneal(qubits,[h,J],T=100,points = 100)

def f4(lb):
    '''This is from Nicks eq 14'''
    N = float(len(lb)); Nup = float(np.sum(lb))
    return Ja*(-2*Nup**2+2*N*Nup+2*Nup-N)-q0*(2*Nup-N)


print 'Nup=1','Nup=3'
print (vo1*a.Hp*vo1.getH()).item(0), (vo2*a.Hp*vo2.getH()).item(0), ' hamiltonian'
print f4(lbo1), f4(lbo2), ' nicks function', '\n'

print'Nup=0','Nup=2','Nup=4'
print (ve1*a.Hp*ve1.getH()).item(0), (ve2*a.Hp*ve2.getH()).item(0), (ve3*a.Hp*ve3.getH()).item(0), ' hamiltonian '
print f4(lbe1), f4(lbe2), f4(lbe3), ' nicks function'

In [ ]:
# test the self consistancy of what is in the paper
Ja = 1.
J = 2.
q0 = 0.5
h = 200.

def eq11(lb):
    '''This is Nicks'''
    N = float(len(lb)); Nup = float(np.sum(lb))
    return 2*Nup-N

def eq12(lb):
    '''This is Nicks'''
    N = float(len(lb)); Nup = float(np.sum(lb))
    return 0.5*(2*Nup-N)**2-0.5*N

def eq13(lb):
    '''This is Nicks'''
    N = float(len(lb)); Nup = float(np.sum(lb))
    return -(2*Nup-N)**2-2*Nup

def eq14(lb):
    '''This is Nicks'''
    N = float(len(lb)); Nup = float(np.sum(lb))
    return Ja*(-2*Nup**2+2*N*Nup+2*Nup-N)-q0*(2*Nup-N)

def eq15(lb):
    '''This is Nicks'''
    N = float(len(lb)); Nup = float(np.sum(lb)); v = 2*Nup-N
    return h*v+J*(0.5*v**2-0.5*N)+Ja*(-0.5*v**2+v-0.5*N**2-N)-q0*v

def eq3(lb):
    '''This is Nicks'''
    return J*eq12(lb)+h*eq11(lb)+Ja*eq13(lb)+eq14(lb)

print 'Nup=1','Nup=3'
print eq15(lbo1), eq15(lbo2), '''Nicks equation 15'''
print eq3(lbo1), eq3(lbo2), 'Nicks equation 3 by parts', '\n'

print'Nup=0','Nup=2','Nup=4'
print eq15(lbe1), eq15(lbe2), eq15(lbe3), 'Nicks equation 15'
print eq3(lbe1), eq3(lbe2), eq3(lbe3),'Nicks equation 3 by parts', '\n'

In [ ]:
Ja = 1.
J = 2.
q0 = 0.5
h = 200.

def myeq11(lb):
    '''This is mine'''
    N = float(len(lb)); Nup = float(np.sum(lb))
    return 2*Nup-N

def myeq12(lb):
    '''This is mine'''
    N = float(len(lb)); Nup = float(np.sum(lb))
    return 0.5*(2*Nup-N)**2-0.5*N

def myeq13(lb):
    '''This is mine'''
    N = float(len(lb)); Nup = float(np.sum(lb))
    return -(2*Nup-N)**2

def myeq14(lb):
    '''This is mine'''
    N = float(len(lb)); Nup = float(np.sum(lb))
    return Ja*(2*Nup**2+2*Nup-N-2*N*Nup)-q0*(2*Nup-N)

def myeq15(lb):
    '''This is mine'''
    N = float(len(lb)); Nup = float(np.sum(lb)); v = 2*Nup-N
    return (h-q0)*v+J*(0.5*v**2-0.5*N)+Ja*(-0.5*v**2-0.5*N**2+v)

def myeq3(lb):
    '''This is mine'''
    return J*myeq12(lb)+h*myeq11(lb)+Ja*myeq13(lb)+myeq14(lb)

print 'Nup=1','Nup=3'
print myeq15(lbo1), myeq15(lbo2), 'my equation 15'
print myeq3(lbo1), myeq3(lbo2), 'my equation 3 by parts', '\n'

print'Nup=0','Nup=2','Nup=4'
print myeq15(lbe1), myeq15(lbe2), myeq15(lbe3), 'my equation 15'
print myeq3(lbe1), myeq3(lbe2), myeq3(lbe3),'my equation 3 by parts', '\n'

### I think I have now got to the point of correcting the equations in Nick's paper. So lets try again and see if we can build the neutral hamiltonian

In [ ]:
log_qub = 4
anc_qub = log_qub
qubits = log_qub*2

# JN<<q0<Ja;   J = Ja

JN = 0.1
q0 = 50.
Ja = 100.
Jl = Ja

hl = np.asarray([-Ja+q0]*log_qub)
ha = np.asarray([-Ja*(2*i-log_qub)+q0 for i in range(1,log_qub+1)])#+np.asarray([JN if (log_qub-i)%2 == 1 else -JN+q0 for i in range(1,log_qub+1)])
# Hia = -Ja*(2*i-N)+qi
h = np.concatenate((hl,ha))

J_log_log = Jl*np.triu(np.ones((log_qub,log_qub)), k=1)
J_log_anc = Ja*np.ones((log_qub,log_qub))
Jt = np.concatenate((J_log_log,J_log_anc),axis=1)
Jb = np.zeros(Jt.shape)
J = np.concatenate((Jt,Jb),axis=0)

lbo1 = [0,0,0,1]
ab = [0]*(sum(lbo1)) + [1]*(len(lbo1)-sum(lbo1))
bitso1 = lbo1+ab
vo1 = mylib.make_eigen_vector(bitso1)

lbo2 = [0,1,1,1]
ab = [0]*(sum(lbo2)) + [1]*(len(lbo2)-sum(lbo2))
bitso2 = lbo2+ab
vo2 = mylib.make_eigen_vector(bitso2)

lbe1 = [0,0,0,0]
ab = [0]*(sum(lbe1)) + [1]*(len(lbe1)-sum(lbe1))
bitse1 = lbe1+ab
ve1 = mylib.make_eigen_vector(bitse1)

lbe2 = [0,0,1,1]
ab = [0]*(sum(lbe2)) + [1]*(len(lbe2)-sum(lbe2))
bitse2 = lbe2+ab
ve2 = mylib.make_eigen_vector(bitse2)

lbe3 = [1,1,1,1]
ab = [0]*(sum(lbe3)) + [1]*(len(lbe3)-sum(lbe3))
bitse3 = lbe3+ab
ve3 = mylib.make_eigen_vector(bitse3)

def myeq15(lb,h,q0,J,Ja):
    '''This is mine'''
    N = float(len(lb)); Nup = float(np.sum(lb)); v = 2*Nup-N
    return (h-q0)*v+J*(0.5*v**2-0.5*N)+Ja*(-0.5*v**2-0.5*N**2+v)

a = mylib.Anneal(qubits,[h,J],T=100,points = 100)

print 'the solution should be %s'%(-Ja/2. * (log_qub**2+log_qub))
print'Nup=1','Nup=3'
print vo1*a.Hp*vo1.getH(), vo2*a.Hp*vo2.getH()
print myeq15(lbo1,hl[0],q0,Jl,Ja), myeq15(lbo2,hl[0],q0,Jl,Ja), 'my equation 15'
print'Nup=0','Nup=2','Nup=4'
print ve1*a.Hp*ve1.getH(), ve2*a.Hp*ve2.getH(), ve3*a.Hp*ve3.getH()
print myeq15(lbe1,hl[0],q0,Jl,Ja), myeq15(lbe2,hl[0],q0,Jl,Ja), myeq15(lbe3,hl[0],q0,Jl,Ja), 'my equation 15'

This above appears to give evidence that at least my derivation of the equations are consistant with my hamiltonian

In [ ]:
print lbe1,bitse1
print lbo1,bitso1
print lbe2,bitse2
print lbo2,bitso2
print lbe3,bitse3


In [ ]:
import scipy as sp
d, P = sp.linalg.eigh(a.Hp)
for i in range(16):
    bitsol = mylib.bit_values(qubits,np.matrix(P[:,i]).getH())
    print 'Nup = %s'%sum(bitsol[:4]),bitsol[4:]

# Thank God, I've done it! Next lets try and build the 4-body interaction Hamiltonian 

In [ ]:
log_qub = 4
anc_qub = log_qub
qubits = log_qub*2

# JN<<q0<Ja;   J = Ja

JN = 0.1
q0 = 50.
Ja = 100.
Jl = Ja

hl = np.asarray([-Ja+q0]*log_qub)
ha = np.asarray([-Ja*(2*i-log_qub)+q0 for i in range(1,log_qub+1)])+np.asarray([JN if (log_qub-i)%2 == 1 else -JN for i in range(1,log_qub+1)])
# Hia = -Ja*(2*i-N)+qi
h = np.concatenate((hl,ha))

J_log_log = Jl*np.triu(np.ones((log_qub,log_qub)), k=1)
J_log_anc = Ja*np.ones((log_qub,log_qub))
Jt = np.concatenate((J_log_log,J_log_anc),axis=1)
Jb = np.zeros(Jt.shape)
J = np.concatenate((Jt,Jb),axis=0)

lbo1 = [0,0,0,1]
ab = [0]*(sum(lbo1)) + [1]*(len(lbo1)-sum(lbo1))
bitso1 = lbo1+ab
vo1 = mylib.make_eigen_vector(bitso1)

lbo2 = [0,1,1,1]
ab = [0]*(sum(lbo2)) + [1]*(len(lbo2)-sum(lbo2))
bitso2 = lbo2+ab
vo2 = mylib.make_eigen_vector(bitso2)

lbe1 = [0,0,0,0]
ab = [0]*(sum(lbe1)) + [1]*(len(lbe1)-sum(lbe1))
bitse1 = lbe1+ab
ve1 = mylib.make_eigen_vector(bitse1)

lbe2 = [0,0,1,1]
ab = [0]*(sum(lbe2)) + [1]*(len(lbe2)-sum(lbe2))
bitse2 = lbe2+ab
ve2 = mylib.make_eigen_vector(bitse2)

lbe3 = [1,1,1,1]
ab = [0]*(sum(lbe3)) + [1]*(len(lbe3)-sum(lbe3))
bitse3 = lbe3+ab
ve3 = mylib.make_eigen_vector(bitse3)

a = mylib.Anneal(qubits,[h,J],T=100,points = 100)

print 'the median energy level is %s'%(-Ja/2. * (log_qub**2+log_qub)-JN*(1-log_qub%2) )
print vo1*a.Hp*vo1.getH(), vo2*a.Hp*vo2.getH()
print ve1*a.Hp*ve1.getH(), ve2*a.Hp*ve2.getH(), ve3*a.Hp*ve3.getH()

I think I have now sorted this out. I'm ready to move on to implement different schedules on each of the classes of qubits